In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE34450"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE34450"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE34450.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE34450.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE34450.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE34450_family.soft.gz', 'GSE34450_series_matrix.txt.gz']
Identified SOFT files: ['GSE34450_family.soft.gz']
Identified matrix files: ['GSE34450_series_matrix.txt.gz']



Background Information:
!Series_title	"Genes Associated with MUC5AC Expression in the Human Airway Epithelium"
!Series_summary	"To help define the genes associated with mucus synthesis and secretion in the human small airway epithelium, we hypothesized that comparison of the transcriptomes of the small airway epithelium of individuals that express high vs low levels of MUC5AC, a major secretory mucin and the major component of airway mucus, could be used as a probe to identify the genes related to human small airway mucus production / secretion.   Genome-wide comparison between healthy nonsmokers grouped as “high MUC5AC expressors” vs “low MUC5AC expressors” identified significantly up-regulated and  down-regulated genes in the high vs low expressors. Based on the literature, genes in the up-regulated list were used to identify a 73 “MUC5AC-associated core gene” list with 9 categories: mucus components; mucus-producing cell differentiation-related transcription factor; mucus-producing

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional, Callable, Dict, Any
import json
import gzip

# 1. Gene Expression Data Availability
# Based on the background information, this dataset studies gene expression in human airway epithelium
# related to MUC5AC expression, which is a mucin gene. This suggests gene expression data is likely available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For Vitamin D levels, we can see row 3 contains "serum 25-oh-d" values (looking at the sample characteristics dictionary)
# 25-OH-D is the standard measurement for vitamin D levels
trait_row = 3  # Using row 3 which contains "serum 25-oh-d" values

# Age data is not available in the sample characteristics
age_row = None  

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert vitamin D level to binary: low (0) vs high/mid (1)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: low vitamin D (0) vs high/mid vitamin D (1)
    if "low vitamin d" in value.lower():
        return 0
    elif "high vitamin d" in value.lower() or "mid vitamin d" in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to numeric"""
    # This function won't be used since age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary: female (0) vs male (1)"""
    # This function won't be used since gender data is not available
    return None

# 3. Save Metadata - Initial filtering
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Load the clinical data from the matrix file
    matrix_file = os.path.join(in_cohort_dir, 'GSE34450_series_matrix.txt.gz')
    
    # Read the matrix file line by line to extract sample characteristic information
    with gzip.open(matrix_file, 'rt') as f:
        clinical_data_lines = []
        reading_characteristics = False
        
        for line in f:
            line = line.strip()
            
            # Start collecting data when we reach the sample characteristics section
            if line.startswith('!Sample_characteristics_ch'):
                reading_characteristics = True
                clinical_data_lines.append(line)
            # Stop when we've moved past the characteristics section
            elif reading_characteristics and not line.startswith('!Sample_characteristics_ch'):
                reading_characteristics = False
                break
    
    # Process the clinical data lines to create a proper DataFrame
    if clinical_data_lines:
        # Extract the values from each line
        processed_lines = []
        for line in clinical_data_lines:
            # Remove the prefix and split by tabs
            values = line.replace('!Sample_characteristics_ch1\t', '').split('\t')
            processed_lines.append(values)
        
        # Transpose the data to have samples as rows and characteristics as columns
        if processed_lines:
            # Ensure all rows have the same length
            max_len = max(len(row) for row in processed_lines)
            for i in range(len(processed_lines)):
                if len(processed_lines[i]) < max_len:
                    processed_lines[i].extend([None] * (max_len - len(processed_lines[i])))
            
            # Create a DataFrame with the transposed data
            clinical_data = pd.DataFrame(processed_lines)
            
            # Extract clinical features
            clinical_df = geo_select_clinical_features(
                clinical_data, 
                trait=trait, 
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the data
            preview = preview_df(clinical_df)
            print("Clinical data preview:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data
            clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Failed to process clinical data lines")
    else:
        print("No sample characteristics found in the matrix file.")


Clinical data preview:
{0: [nan], 1: [1.0], 2: [nan], 3: [nan], 4: [nan], 5: [nan], 6: [nan], 7: [nan], 8: [nan], 9: [nan], 10: [nan], 11: [1.0], 12: [1.0], 13: [1.0], 14: [nan], 15: [1.0], 16: [nan], 17: [nan], 18: [1.0], 19: [nan], 20: [nan], 21: [0.0], 22: [1.0], 23: [0.0], 24: [nan], 25: [nan], 26: [1.0], 27: [nan], 28: [nan], 29: [nan], 30: [1.0], 31: [nan], 32: [1.0], 33: [nan], 34: [nan], 35: [nan], 36: [nan], 37: [nan], 38: [nan], 39: [nan], 40: [nan], 41: [nan], 42: [nan], 43: [nan], 44: [nan], 45: [nan], 46: [nan], 47: [nan], 48: [nan], 49: [nan], 50: [nan], 51: [nan], 52: [nan], 53: [nan], 54: [nan], 55: [nan], 56: [nan], 57: [nan], 58: [nan], 59: [nan], 60: [nan], 61: [nan], 62: [nan], 63: [nan], 64: [nan], 65: [nan], 66: [nan], 67: [nan], 68: [nan], 69: [nan], 70: [nan], 71: [nan], 72: [nan], 73: [nan], 74: [nan], 75: [nan], 76: [nan], 77: [nan], 78: [nan], 79: [nan], 80: [nan], 81: [nan], 82: [nan], 83: [nan], 84: [nan], 85: [nan], 86: [nan], 87: [nan], 88: [nan], 89: [na

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data shape: (54675, 132)


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# Based on biomedical knowledge, these identifiers appear to be Affymetrix probe IDs (e.g., "1007_s_at")
# rather than standard human gene symbols (which would be like "BRCA1", "TP53", etc.)
# These probe IDs need to be mapped to official gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
             GSM549681  GSM549682   GSM549683  GSM549684  GSM549685
ID                                                                 
1007_s_at  16230.50000  7351.5600  5082.78000  5892.5400  7686.6100
1053_at      433.80300   297.6980   130.80100   289.7050   312.6770
117_at        67.07860    52.8227    87.65820   419.4480    49.5630
121_at       871.85000   512.4560   376.08700   453.4440   370.7270
1255_g_at      7.74082    58.1624     6.89994    11.3485    19.2553



Platform information:
!Series_title = Genes Associated with MUC5AC Expression in the Human Airway Epithelium
!Platform_title = [HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = June 03, 2009: annotation table updated with netaffx build 28
!Platform_description = June 06, 2012: annotation table updated with netaffx build 32
!Platform_description = June 23, 2016: annotation table updated with netaffx build 35
#Target Description =
#RefSeq Transcript ID = References to multiple sequences in RefSeq. The field contains the ID and Description for each entry, and there can be multiple entries per ProbeSet.
#Gene Ontology Biological Process = Gene Ontology Consortium Biological Process derived from LocusLink.  Each annotation consists of three parts: "Accessi


Gene annotation columns:
['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']

Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation data, determine mapping columns
# The probe IDs in gene_data match with the 'ID' column in gene_annotation
# The gene symbols are in the 'Gene Symbol' column
gene_id_col = 'ID'
gene_symbol_col = 'Gene Symbol'

# 2. Create a mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to official HGNC symbols where possible
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (45782, 2)
First few rows of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Mapped gene expression data shape: (21278, 132)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')
Gene expression data shape after normalization: (19845, 132)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Vitamin_D_Levels/gene_data/GSE34450.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (19845, 132)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Vitamin_D_Levels/gene_data/GSE34450.csv
Loaded clinical data:
       1   2   3   4   5   6   7   8   9  10  ...  122  123  124  125  126  \
0                                             ...                            
NaN  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN  NaN  NaN  NaN  NaN   

     127  128  129  130  131  
0                             
NaN  NaN  NaN  NaN  NaN  NaN  

[1 rows x 131 columns]
Transposed clinical data to correct format:
0  NaN
1  1.0
2  NaN
3  NaN
4  NaN
5  NaN
Number of common samples between clinical and genetic data: 0
Clinical data index: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',

Linked data shape after handling missing values: (132, 19846)
For the feature 'Vitamin_D_Levels', the least common label is '1' with 14 occurrences. This represents 10.61% of the dataset.
The distribution of the feature 'Vitamin_D_Levels' in this dataset is fine.

Is trait biased: False
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Vitamin_D_Levels/GSE34450.csv
